<a name="top"></a>
<br/>
# Understanding how `medGAN` works on the MIMIC-III dataset of shape `(46520, 1071)` with binary values

Author: [Sylvain Combettes](https://github.com/sylvaincom). <br/>
Last update: Sep 4, 2019. Creation: Aug 12, 2019. <br/>
My own medGAN repository (that is based on Edward Choi's work): [medgan-tips](https://github.com/sylvaincom/medgan-tips). <br/>
Edward Choi's original repository: [medgan](https://github.com/mp2893/medgan).

The final goal of my project is to use `medGAN` on my own dataset (electronic health records). Hence, I first need to understand how the `medGAN` program works. In this notebook, I provide a few code cells and explanations to help better understand and run `medGAN`. Because there are some confidentiality issues with the MIMIC-III dataset, I cleared the output of the cells.

Before reading this notebook, make sure that you have read my [medGAN repository](https://github.com/sylvaincom/medgan-tips)'s table of contents.

We will use the MIMIC-III dataset and process it so that we only have binary values.

---
### Table of contents

- [1) Loading the MIMIC-III dataset](#load-mimic)
- [2) Using `process_mimic.py` and `medgan.py` to generate the fake realistic data](#run)
- [3) How can one interpret the output of `medgan.py`?](#gen-samples)
- [4) How can one interpret the output of `process_mimic.py`?](#input)
- [5) Comparing the fictitious generated samples to the real-life original ones](#comparison)

---
### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import pickle

---
# 1) Loading the MIMIC-III dataset <a name="load-mimic"></a>

## 1.1) `ADMISSIONS.csv` file

In [ ]:
df_adm = pd.read_csv("ADMISSIONS.csv")
print(df_adm.shape)
df_adm.head()

Do we have a lot of missing values?

In [ ]:
def observe_missing_values(df):
    # This function does not return any values.
    
    dico = {}
    c_index = [] # index of columns of features with missing values
    n,p = df.shape
    features = list(df) # list of features
    
    # We compute the percentage of missing values for each column:
    for f in features:
        percentage = round(sum(df[f].isna())*100/n, 2) # percentage of missing values
        if percentage>0:
            dico[f] = [percentage]
            c_index.append(df.columns.get_loc(f))
    
    if c_index == []:
        return 'There are no missing values!'
    
    # We construct the DataFrame ordered by Approx. missing values (%):
    df_mv = pd.DataFrame(data=dico) # mv for missing values
    idx_rename = {df_mv.index.tolist()[0]:'Approx. missing values (%)'}
    df_mv = df_mv.rename(index=idx_rename)
    
    # We print the features with a decreasing Approx. missing values (%) value:
    n_f = min(p,10) # number of features we choose to print
    df_mv_ordered = df_mv.sort_values(by=['Approx. missing values (%)'], axis=1, ascending=False)
    print('The', n_f, 'features with the most missing values are:')
    print(df_mv_ordered.iloc[0].head(n_f))
    
    # We plot the Approx. missing values (%) given the index of the feature:
    plt.plot(c_index, df_mv.iloc[0], 'o')
    plt.xlabel('Index of feature')
    plt.ylabel('Approx. missing values (%)')
    plt.title('Observing missing values')
    plt.show()

In [ ]:
observe_missing_values(df_adm)

## 1.2) `DIAGNOSES_ICD.csv` file

In [ ]:
df_ICD = pd.read_csv('DIAGNOSES_ICD.csv')
print(df_ICD.shape)
df_ICD.head()

Do we have a lot of missing values?

In [ ]:
observe_missing_values(df_ICD)

We check if our dataset is balanced. Does one `ICD9_CODE` appear distinctly more than others in proportion?

In [ ]:
df_ICD['ICD9_CODE'].value_counts(normalize=True).head()

---
# 2) Using `process_mimic.py` and `medgan.py` to generate the fake realistic data <a name="run"></a>

This step is detailed in [A few additional tips on how to run Edward Choi's medGAN](https://github.com/sylvaincom/medgan/blob/master/tips-for-medgan.md).

In short, in the Anaconda prompt, we run:
```
cd C:\Users\<username>\Documents\mimic_binary
python process_mimic.py ADMISSIONS.csv DIAGNOSES_ICD.csv training-data "binary"
mkdir generated
python medgan.py training-data.matrix ./generated/samples --data_type="binary"
python medgan.py training-data.matrix gen-samples --model_file=./generated/samples-999 --generate_data=True --data_type="binary"
```
Some default values are `n_epoch=1000`, `n_pretrain_epoch=100` and `batch_size=1000`. We choose `nSamples=10000` line 405 in `medgan.py`.
The computing took more than 5 hours on my laptop.

---
# 3) How can one interpret the output of `medgan.py`? <a name="gen-samples"></a>

We load the `gen-samples.npy` file which is `medgan.py`'s output:

In [ ]:
fict = np.load('gen-samples.npy') # fictitious generated dataset
df_fict = pd.DataFrame(fict)
print(df_fict.shape)
df_fict.head()

Do the output of `medgan.py` have missing values?

In [ ]:
observe_missing_values(df_fict)

The output of `medgan.py` has no missing values!

Some questions about this data frame:
* What do the columns correspond to? They are not the ones of `ADMISSIONS.csv` nor `DIAGNOSIS_ICD.csv`.
* What do the rows correspond to?
* Why are the values not binary?

We can find some answers in an issue opened in Edward Choi's GitHub: [How to interpret the samples?](https://github.com/mp2893/medgan/issues/3). In order to understand the output `gen-samples.npy` of `medgan.py`, we are going to go back to the input of `medgan.py`: the output of `process_mimic.py` which is `training-data.matrix`.

Actually, in `gen-samples.npy`, as in the `training-data.matrix` file, each row corresponds to a single patient and each column corresponds to a specific ICD9 diagnosis code. We can use the `training-data.types` file created by `process_mimic.py` to map each column to a specific ICD9 diagnosis code. Read the beginning part of the source code of `process_mimic.py` for more information about these files:
```python
# Output files
# <output file>.pids: cPickled Python list of unique Patient IDs. Used for intermediate processing
# <output file>.matrix: Numpy float32 matrix. Each row corresponds to a patient. Each column corresponds to a ICD9 diagnosis code.
# <output file>.types: cPickled Python dictionary that maps string diagnosis codes to integer diagnosis codes.
```

What is ICD-9? See [ICD-9](https://en.wikipedia.org/wiki/International_Statistical_Classification_of_Diseases_and_Related_Health_Problems#ICD-9) and [List of ICD-9 codes](https://en.wikipedia.org/wiki/List_of_ICD-9_codes).

We need to round the values ourselves:

In [ ]:
df_fict = df_fict.round(0)
df_fict.head()

Indeed, in line 405 of `medgan.py`, we have chosen `nSamples=10000`. It is important to note thay once we have learned the estimated density of the real-life original samples, we can generate as many fake realistic samples as we want.

---
# 4) How can one interpret the output of `process_mimic.py`? <a name="input"></a>

## 4.1) Understanding `training-data.types`

_cPickled Python dictionary that maps string diagnosis codes to integer diagnosis codes._

In [ ]:
map_dict = pickle.load(open('training-data.types', 'rb'))
print(type(map_dict))
print('An excerpt is:', dict(list(map_dict.items())[0:5]))

Thus, as its name suggests, `process_mimic.py` is really dependent on the MIMIC-III dataset. We probably will not use `process_mimic.py` on our own dataset and only run `medgan.py`. Out of `process_mimic.py`, we only need to understand how the output file `training-data.matrix` is constructed.

## 4.2) Understanding `training-data.pids`

_cPickled Python list of unique Patient IDs. Used for intermediate processing_

In [ ]:
id_list = pickle.load(open('training-data.pids', 'rb'))
print(type(id_list))
print('An excerpt is:', id_list[:10])

## 4.3) Understanding `training-data.matrix`

_Numpy float32 matrix. Each row corresponds to a patient. Each column corresponds to a ICD9 diagnosis code._

`training-data.matrix` is an output of `process_mimic.py` and the input of `medgan.py`.

In [ ]:
real_data_array = pickle.load(open('training-data.matrix', 'rb'))
print(type(real_data_array))
real_data_array

In [ ]:
df_real = pd.DataFrame(real_data_array)
print(df_real.shape)
df_real.head(5)

As we chose, the real-life data is binary. 

We can note that the input and the [output](#gen-samples) of `medgan.py` have the same number of columns and that the values are of the same type (binary). Thus, `gen-samples.npy` is a fictitious realistic generated dataset that is trying to look like the `training-data.matrix` file.

Does the input of `medgan.py` have missing values?

In [ ]:
observe_missing_values(df_real)

The input of `medgan.py` has no missing values!

---
# 5) Comparing the fictitious generated samples to the real-life original ones  <a name="comparison"></a>

In this section, we wish to compare the accuracy of the fictitious generated dataset considering the original one. As in Choi's paper, we use dimension-wise probability (because the variables are binary).

Given that our data is binary, for each feature (dimension), we claim that `1` corresponds to success and `0` to failure. Hence the proportion of `1` obtained is the Bernoulli success probability _p_.

## 5.1) Probability distribution of the real-life data

In [ ]:
df_real = df_real
n_real, p_real = df_real.shape
print(n_real, p_real)

proba_real = [sum(df_real[f])/n_real for f in list(df_real)]

plt.plot(proba_real, 'o')
plt.xlabel('Index of feature')
plt.ylabel('Bernoulli probability success')
plt.title('For the real-life dataset')
plt.show()

## 5.2) Probability distribution of the fictitious generated data

In [ ]:
n_fict, p_fict = df_fict.shape
print(n_fict, p_fict)

proba_fict = [sum(df_fict[f])/n_fict for f in list(df_fict)]

plt.plot(proba_fict, 'o')
plt.xlabel('Index of feature')
plt.ylabel('Bernoulli probability success')
plt.title('For the fictitious generated dataset')
plt.show()

## 5.3) Comparison: dimension-wise probability

In [ ]:
xaxis = proba_real
yaxis = proba_fict

start = min(np.min(xaxis), np.min(yaxis))
stop = max(np.max(xaxis), np.max(yaxis))
p = len(xaxis)
X = np.linspace(start, stop, num=p+1)

plt.plot(xaxis, yaxis, 'ok', X, X, '-g');

plt.legend(['Bernoulli success probability', 'ideal Bernoulli success probability'])
plt.title('Dimension-wise probability performance of medGAN')
plt.xlabel('For the real dataset')
plt.ylabel('For the fictitious generated dataset')
plt.savefig('accuracy_mimic_binary.png', dpi=120) # to save the image in high resolution
plt.show()

The values of the $x$-axis and the $y$-axis are ordered: we successively compare the Bernoulli success probability of both datasets for a given variable.

For information, we have 1 071 features thus 1 071 scatter points. Note that once we have learned the distribution of our real-life original data, we can generate as many (fake) samples as we want, for example 10 000.

The diagonal green line indicates the ideal performance where the real and the fictitious realistic generated data show identical quality. Based on the graph, as the dots are close to the diagonal green line, we can say that `medGAN` has a really good accuracy.

---
Back to [top](#top).